# 02_Environment_Development

Construct and test the custom multi-asset RL environment.


In [ ]:
# 2.1 Install & Imports
%pip install -e ..
%pip install gymnasium stable-baselines3

import sys
sys.path.append('../src')

import gymnasium as gym
import numpy as np
import pandas as pd
from src.environment.Multi_asset_env import MultiAsset21DeepHedgingEnv
from src.environment.option_pricing import create_synthetic_option_chain
from src.utils.data_utils import download_market_data
from src.config.settings import get_config


In [ ]:
# 2.2 Prepare Data & Option Chain
cfg = get_config('data')
df = download_market_data(
    symbol=cfg['symbol'],
    start_date=cfg['start_date'],
    end_date=cfg['end_date'],
    interval=cfg['interval']
)

opt_chain = create_synthetic_option_chain(
    spot_data=df,
    config=get_config('option')
)

# Define asset universe (same as in README)
strikes = get_config('option')['strike_offsets']
expiries = get_config('option')['expiry_days']
types_ = get_config('option')['option_types']
asset_universe = [
    {'strike_offset': s, 'expiry_days': e, 'type': t}
    for e in expiries for s in strikes for t in types_
]


## 2.3 Initialize Environment and Test State


In [ ]:
env = MultiAsset21DeepHedgingEnv(df, opt_chain, asset_universe)
state, _ = env.reset()
print("Observation shape:", state.shape)
print("Action space:", env.action_space)
print("Observation sample:", state[:10])


## 2.4 Step Through Environment


In [ ]:
action = env.action_space.sample()
next_state, reward, done, trunc, info = env.step(action)
print("Next state shape:", next_state.shape)
print("Reward:", reward)
print("Done:", done)


## 2.5 Visualize a Random Episode


In [ ]:
obs, _ = env.reset()
rewards = []
for _ in range(50):
    act = env.action_space.sample()
    obs, r, done, trunc, _ = env.step(act)
    rewards.append(r)
    if done: break

import matplotlib.pyplot as plt
plt.plot(rewards)
plt.title("Random Episode Rewards (50 steps)")
plt.xlabel("Step")
plt.ylabel("Reward")
plt.show()
